In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.rcParams['image.cmap'] = 'gist_earth'
np.random.seed(98765)

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.gpu_device_name()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 265812221517771837
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15862461236
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2065933170674148190
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:0f:00.0, compute capability: 6.0"
]


'/device:GPU:0'

In [3]:
import sys
# Add the tf_unet folder path to the sys.path list
sys.path.append('/data3/floraxue/cs282a/SegmentNet/')


In [4]:
from tf_unet import image_gen
from tf_unet import unet
from tf_unet import util, image_util

In [5]:
#Data input
#put your data here
path_images = "/data3/floraxue/cs282a/data/dataset_1/trainval/M_phase_full/"
generator = image_util.ImageDataProvider(path_images+"*.tif")
#TODO: need to enhance the borders and rethink the matrix vs bkg 

Number of files used: 100
Number of channels: 1
Number of classes: 2


In [8]:
net = unet.Unet(channels=1, n_class=2, layers=2, features_root=16)

2019-05-04 19:14:22,318 Layers 2, features 16, filter size 3x3, pool size: 2x2


In [9]:
trainer = unet.Trainer(net, batch_size=1, verification_batch_size = 1, 
                       optimizer="adam", opt_kwargs=dict(learning_rate = 0.0001))

In [10]:
path = trainer.train(generator, "./unet_M_adam_full", training_iters=100, epochs=5, display_step=40)

2019-05-04 19:14:25,313 Removing '/data3/floraxue/cs282a/SegmentNet/M_net/prediction'
2019-05-04 19:14:25,322 Removing '/data3/floraxue/cs282a/SegmentNet/M_net/unet_M_adam_full'
2019-05-04 19:14:25,328 Allocating '/data3/floraxue/cs282a/SegmentNet/M_net/prediction'
2019-05-04 19:14:25,333 Allocating '/data3/floraxue/cs282a/SegmentNet/M_net/unet_M_adam_full'
2019-05-04 19:14:30,590 Verification error= 50.8%, loss= 0.6935
2019-05-04 19:14:33,575 Start optimization
2019-05-04 19:14:43,737 Iter 0, Minibatch Loss= 0.6909, Training Accuracy= 0.5172, Minibatch error= 48.3%
2019-05-04 19:15:50,828 Iter 40, Minibatch Loss= 0.6104, Training Accuracy= 0.9767, Minibatch error= 2.3%
2019-05-04 19:16:56,622 Iter 80, Minibatch Loss= 0.4877, Training Accuracy= 0.9718, Minibatch error= 2.8%
2019-05-04 19:17:21,735 Epoch 0, Average loss: 0.5660, learning rate: 0.0001
2019-05-04 19:17:22,536 Verification error= 2.6%, loss= 0.3156
2019-05-04 19:17:50,912 Iter 120, Minibatch Loss= 0.1429, Training Accuracy

In [ ]:
from skimage import io
x_test = io.imread(path_images+'0000.tif')
y_test = io.imread(path_images+'0000_mask.tif')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")

In [ ]:
x_test.shape

In [ ]:
#x_test, y_test = generator(1)
h,w = x_test.shape
x = x_test.reshape(1,h,w,1)
prediction = net.predict("./unet_trained/model.ckpt", x)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")
mask = prediction[0,...,1] > 0.1
ax[2].imshow(mask, aspect="auto")
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")

In [ ]:
fig.tight_layout()
fig.savefig("../docs/toy_problem.png")